In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv('movies.csv')

#sets movieId as the key
movies.set_index('movieId', inplace=True)

#converts genre into a list of genres while returning no genres as an empty list
movies['genres'] = movies['genres'].apply(lambda x: x.split('|') if x != '(no genres listed)' else [])

#creates column for year by extracting from title
movies['year'] = movies['title'].str.extract(r'\((\d{4})\)')

movies['freq'] = 0








In [4]:
#ratings.csv is too large for github it can be found online here https://www.kaggle.com/datasets/parasharmanas/movie-recommendation-system/data
ratings = pd.read_csv('ratings.csv')

In [5]:
#cast ids to ints
ratings['userId'] = ratings['userId'].astype(int)
ratings['movieId'] = ratings['movieId'].astype(int)

In [6]:
max_userId = ratings['userId'].max()
print(movies.columns)
max_movieId = movies.index.max()
num_users = 10000
user_rating_matrix = np.zeros((num_users + 1, max_movieId + 1))



Index(['title', 'genres', 'year', 'freq'], dtype='object')


In [13]:

for index, row in ratings.iterrows():
    userId = row['userId']
    if userId > num_users:
        break
    movieId = row['movieId']
    rating = row['rating']
    user_rating_matrix[int(userId)][int(movieId)] = rating
    movies.loc[int(movieId), 'freq'] += 1
       

In [15]:
movies.to_csv('movies_1.0.csv')

In [ ]:
import numpy as np
from sklearn.decomposition import TruncatedSVD

n_latent_factors = 5 

svd = TruncatedSVD(n_components=n_latent_factors)
svd.fit(user_rating_matrix)

# Extract user (P) and movie (Q) latent factors
P = svd.transform(user_rating_matrix)
Q = svd.components_.T



In [ ]:
print(Q.shape)

(209172, 10)


In [ ]:
import csv
csv_file = "movie_latent_factors_1.0.csv"
with open(csv_file, mode='w', newline='') as file:
    writer = csv.writer(file)

    # Write data rows with row labels
    for i, row in enumerate(Q):
        writer.writerow(row)
    
print(f"CSV data has been written to {csv_file}")

CSV data has been written to movie_latent_factors_1.0.csv


In [ ]:
from numpy.linalg import norm
def cosine_sim(vec1, vec2):
  sim = 0
  if norm(vec1) != 0 and norm(vec2) != 0:
    sim = np.dot(vec1, vec2) / (norm(vec1) * norm(vec2))
  return sim

sim_list = []
movie_list = []
id = 2194
lat_1 = Q[id]

for i in range(len(Q)):
    if(i != id):
        lat_2 = Q[i]
        sim = cosine_sim(lat_1, lat_2)
        sim_list.append(sim)
        movie_list.append(i)

combined_lists = list(zip(sim_list, movie_list))
sorted_combined_lists = sorted(combined_lists, key=lambda x: x[0], reverse = True)
sim_list, movie_list = zip(*sorted_combined_lists)

print("The 5 highest ranked for the movie,",movies.loc[id]['title'],", are:")

for i in range(5):
    print(sim_list[i], '|', movies.loc[movie_list[i]]['title'])




The 5 highest ranked for the movie, Untouchables, The (1987) , are:
0.9697128554305895 | Rocky (1976)
0.9506571568954755 | Platoon (1986)
0.9342225949087819 | Hunt for Red October, The (1990)
0.9107298656564338 | Lethal Weapon (1987)
0.8677048205886808 | Midnight Run (1988)
